# 3 - Select columns, filter onshore

In [1]:
import pandas as pd
import numpy as np
from datetime import date

today = date.today().isoformat()

In [2]:
import rpy2.rinterface

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
suppressMessages(library(tidyverse))

## 3.1 Extract relevant columns of the pipeline incidents dataset

In [5]:
incidents = pd.read_excel('../data/incidents_2019-08-01/hl2010toPresent.xlsx', 
                          sheet_name=1)
incidents_selected = incidents[['OPERATOR_ID', 'LOCAL_DATETIME', 'NAME', 'COMMODITY_RELEASED_TYPE', 
                                'SERIOUS', 'SIGNIFICANT', 'LOCATION_LATITUDE', 'LOCATION_LONGITUDE', 
                                'ON_OFF_SHORE']].copy()

incidents_selected.sample(5)

,OPERATOR_ID,LOCAL_DATETIME,NAME,COMMODITY_RELEASED_TYPE,SERIOUS,SIGNIFICANT,LOCATION_LATITUDE,LOCATION_LONGITUDE,ON_OFF_SHORE
2937,39205,2017-05-04 13:30:00,"DAPL-ETCO OPERATIONS MANAGEMENT, LLC",CRUDE OIL,NO,NO,35.403170,-89.528190,ONSHORE
2181,2552,2015-08-04 05:10:00,COLONIAL PIPELINE CO,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,NO,NO,30.528648,-91.746802,ONSHORE
3003,30782,2017-07-29 17:45:00,HARVEST PIPELINE COMPANY,CRUDE OIL,NO,NO,29.896540,-92.069790,ONSHORE
3208,31570,2018-01-03 09:00:00,TESORO HIGH PLAINS PIPELINE COMPANY LLC,CRUDE OIL,NO,NO,48.288314,-102.924450,ONSHORE
1385,32009,2013-10-14 13:27:00,EXXONMOBIL OIL CORPORATION-TERMINALS,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,NO,YES,33.866570,-117.838615,ONSHORE


In [6]:
import numpy as np

np.unique(incidents_selected['COMMODITY_RELEASED_TYPE'])

array(['BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL BLENDS)',
       'CO2 (CARBON DIOXIDE)', 'CRUDE OIL',
       'HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS',
       'REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS'],
      dtype=object)

In [7]:
incidents_selected['ON_OFF_SHORE'].value_counts()

ONSHORE     3791
OFFSHORE      28
Name: ON_OFF_SHORE, dtype: int64

### 3.1.1 Fix data types

In [8]:
incidents_selected.dtypes

OPERATOR_ID                         int64
LOCAL_DATETIME             datetime64[ns]
NAME                               object
COMMODITY_RELEASED_TYPE            object
SERIOUS                            object
SIGNIFICANT                        object
LOCATION_LATITUDE                 float64
LOCATION_LONGITUDE                float64
ON_OFF_SHORE                       object
dtype: object

In [9]:
incidents_selected['OPERATOR_ID'] = incidents_selected['OPERATOR_ID'].astype(str)
incidents_selected.dtypes

OPERATOR_ID                        object
LOCAL_DATETIME             datetime64[ns]
NAME                               object
COMMODITY_RELEASED_TYPE            object
SERIOUS                            object
SIGNIFICANT                        object
LOCATION_LATITUDE                 float64
LOCATION_LONGITUDE                float64
ON_OFF_SHORE                       object
dtype: object

Make sure SERIOUS and SIGNIFICANT are booleans.

In [10]:
(incidents_selected[['SERIOUS']] == 'YES')['SERIOUS'].value_counts()

False    3803
True       16
Name: SERIOUS, dtype: int64

In [11]:
incidents_selected['SERIOUS'] = incidents_selected[['SERIOUS']] == 'YES'

In [12]:
(incidents_selected[['SIGNIFICANT']] == 'YES')['SIGNIFICANT'].value_counts()

False    2364
True     1455
Name: SIGNIFICANT, dtype: int64

In [13]:
incidents_selected['SIGNIFICANT'] = incidents_selected[['SIGNIFICANT']] == 'YES'

In [14]:
incidents_selected.dtypes

OPERATOR_ID                        object
LOCAL_DATETIME             datetime64[ns]
NAME                               object
COMMODITY_RELEASED_TYPE            object
SERIOUS                              bool
SIGNIFICANT                          bool
LOCATION_LATITUDE                 float64
LOCATION_LONGITUDE                float64
ON_OFF_SHORE                       object
dtype: object

### 3.1.2 Recode on/offshore to boolean, fix column names

In [15]:
incidents_selected['ONSHORE'] = incidents_selected[['ON_OFF_SHORE']] == 'ONSHORE'
incidents_selected = incidents_selected.drop(columns=['ON_OFF_SHORE'])
incidents_selected = incidents_selected.rename(columns={'COMMODITY_RELEASED_TYPE': 'COMMODITY'})

incidents_selected.sample(5)

,OPERATOR_ID,LOCAL_DATETIME,NAME,COMMODITY,SERIOUS,SIGNIFICANT,LOCATION_LATITUDE,LOCATION_LONGITUDE,ONSHORE
1634,22442,2014-04-24 08:15:00,WEST TEXAS GULF PIPELINE CO,CRUDE OIL,False,False,30.837090,-95.173310,True
2582,300,2016-07-14 13:13:00,"PLAINS PIPELINE, L.P.",CRUDE OIL,False,True,32.008424,-102.013542,True
585,2731,2011-09-08 07:50:00,CHEVRON PIPE LINE CO,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,False,True,32.251168,-101.305851,True
3517,39138,2018-10-24 08:30:00,"BOARDWALK LOUISIANA MIDSTREAM, LLC",HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,False,True,30.250328,-93.405692,True
2954,18092,2017-05-19 07:10:00,"SFPP, LP",REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,False,True,33.846507,-118.232718,True


In [16]:
incidents_selected.to_feather(f'../preprocessed_data/incidents_selected_{today}.feather')

## 3.2 Extract relevant columns of the pipeline system dataset (2010-)

### 3.2.1 Select relevant columns

In [17]:
from os import listdir

pipelines_2010_present = [file for file in listdir('../data/pipelines_2010_present_2019-08-02/') if 'annual_hazardous_liquid' in file]
pipelines_2010_present = pd.concat([pd.read_excel(f'../data/pipelines_2010_present_2019-08-02/{file}', skiprows=2) 
                                    for file in pipelines_2010_present])
pipelines_2010_present = pipelines_2010_present.reset_index(drop = True)

pipelines_2010_present.sample(5)

,DATAFILE_AS_OF,REPORT_YEAR,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,OPERATOR_ID,PARTA2NAMEOFCOMP,PARTA4STREET,PARTA4CITY,PARTA4STATE,PARTA4ZIP,...,PARTE2010HF,PARTE2010LF,PARTE2010TOTAL,PARTETOTAL,PARTETOTALHF,PARTETOTALLF,REPORT_SUBMISSION_TYPE,REPORT_DATE,FILING_DATE,FORM_REV
1232,2019-07-01 11:03:57,2014,20151418,13544,4906,EXXONMOBIL PIPELINE CO,22777 SPRINGWOODS VILLAGE PARKWAY E3.5A.385,SPRING,TX,77389,...,2.47,0.0,2.47,388.43,122.11,266.32,INITIAL,2015-06-11 16:10:42,2015-06-11 16:10:42,7000-1.1 (Rev. 06-2014)
1047,2019-07-01 10:36:07,2011,20120141,11173,32062,IMTT-PIPELINE,250 East 22nd St.,NEW ORLEANS,NJ,07002,...,0.00,0.0,0.00,0.00,0.00,0.00,INITIAL,2012-06-08 08:43:22,2012-06-08 08:43:22,7000-1.1 (Rev. 06-2011)
4198,2019-07-01 11:21:18,2016,20172610,14982,30003,HOUSTON REFINING LP.,12000 LAWNDALE,HOUSTON,TX,77017,...,0.00,0.0,0.00,3.00,3.00,0.00,SUPPLEMENTAL,2017-06-12 12:15:33,2017-06-12 12:49:59,7000-1.1 (Rev. 06-2014)
5247,2019-07-01 11:08:29,2015,20161930,14167,39228,TESORO LOGISTICS ROCKIES,19100 RIDGEWOOD PARKWAY,SAN ANTONIO,TX,78259,...,3.56,0.0,3.56,51.65,51.65,0.00,INITIAL,2016-06-08 14:26:01,2016-06-08 14:26:01,7000-1.1 (Rev. 06-2014)
2044,2019-07-01 10:43:40,2012,20130433,12246,31485,ENBRIDGE OFFSHORE (GAS GATHERING) L.L.C.,"1100 LOUISIANA, SUITE 3300",HOUSTON,TX,77002,...,0.00,0.0,0.00,25.69,25.69,0.00,INITIAL,2013-06-14 18:56:36,2013-06-14 18:56:36,7000-1.1 (Rev. 06-2011)


In [18]:
pipelines_2010_selected = pipelines_2010_present[[
    'OPERATOR_ID', 'REPORT_YEAR', 'PARTA2NAMEOFCOMP', 'PARTA5COMMODITY', 'PARTBHCAONSHORE', 
    'PARTEUNKNTOTAL', 'PARTEPRE40TOTAL', 'PARTE1940TOTAL', 'PARTE1950TOTAL', 'PARTE1960TOTAL', 
    'PARTE1970TOTAL', 'PARTE1980TOTAL', 'PARTE1990TOTAL', 'PARTE2000TOTAL', 'PARTE2010TOTAL',
    'PARTBHCAOFFSHORE', 'PARTBHCATOTAL']].copy()

pipelines_2010_selected.sample(5)

,OPERATOR_ID,REPORT_YEAR,PARTA2NAMEOFCOMP,PARTA5COMMODITY,PARTBHCAONSHORE,PARTEUNKNTOTAL,PARTEPRE40TOTAL,PARTE1940TOTAL,PARTE1950TOTAL,PARTE1960TOTAL,PARTE1970TOTAL,PARTE1980TOTAL,PARTE1990TOTAL,PARTE2000TOTAL,PARTE2010TOTAL,PARTBHCAOFFSHORE,PARTBHCATOTAL
2340,3156,2013,DAVIS GAS PROCESSING,HVL,7.900,0.00,0.00,0.0,0.00,0.000,28.610,0.000,0.000,0.00,2.59,NaN,7.900
2884,1845,2010,"BUCKEYE PARTNERS, LP",HVL,47.691,0.00,0.00,0.0,0.00,0.616,22.841,6.556,17.727,0.00,0.00,NaN,47.691
5002,31847,2015,"CALUMET LUBRICANTS CO., L. P.",Refined and/or Petroleum Product (non-HVL),3.000,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0.000,0.00,0.00,NaN,3.000
3960,39648,2018,VALERO HOUSTON REFINERY,Crude Oil,1.400,0.00,0.00,0.0,1.24,0.160,0.000,0.000,0.000,0.00,0.00,NaN,1.400
1315,22830,2014,WOLVERINE PIPELINE CO,Refined and/or Petroleum Product (non-HVL),513.890,151.72,81.84,0.0,7.58,119.700,242.150,2.690,6.970,38.11,2.08,NaN,513.890


In [19]:
pipelines_2010_selected.dtypes

OPERATOR_ID           int64
REPORT_YEAR           int64
PARTA2NAMEOFCOMP     object
PARTA5COMMODITY      object
PARTBHCAONSHORE     float64
PARTEUNKNTOTAL      float64
PARTEPRE40TOTAL     float64
PARTE1940TOTAL      float64
PARTE1950TOTAL      float64
PARTE1960TOTAL      float64
PARTE1970TOTAL      float64
PARTE1980TOTAL      float64
PARTE1990TOTAL      float64
PARTE2000TOTAL      float64
PARTE2010TOTAL      float64
PARTBHCAOFFSHORE    float64
PARTBHCATOTAL       float64
dtype: object

In [20]:
pipelines_2010_selected['OPERATOR_ID'] = pipelines_2010_selected['OPERATOR_ID'].astype(str)
pipelines_2010_selected['OPERATOR_ID'].dtype

dtype('O')

In [21]:
pipelines_2010_selected = pipelines_2010_selected.rename(
    columns={'REPORT_YEAR': 'YEAR', 'PARTA2NAMEOFCOMP': 'NAME', 'PARTA5COMMODITY': 'COMMODITY', 'PARTBHCAONSHORE': "MILES", 
             'PARTBHCAOFFSHORE': 'OFFSHORE_MILES', 'PARTBHCATOTAL': 'TOTAL_MILES', 'PARTEUNKNTOTAL': 'AGE_UNKNOWN_MILES', 
             'PARTEPRE40TOTAL': 'MILES_PRE_1940', 'PARTE1940TOTAL': 'MILES_1940', 'PARTE1950TOTAL': 'MILES_1950', 
             'PARTE1960TOTAL': 'MILES_1960', 'PARTE1970TOTAL': 'MILES_1970', 'PARTE1980TOTAL': 'MILES_1980', 
             'PARTE1990TOTAL': 'MILES_1990', 'PARTE2000TOTAL': 'MILES_2000', 'PARTE2010TOTAL': 'MILES_2010'})
pipelines_2010_selected.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES
1317,22855,2014,"KOCH PIPELINE COMPANY, L.P.",Fuel Grade Ethanol (dedicated system),1.06,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,1.06
5053,32114,2015,COPANO NGL SERVICES LLC,HVL,75.18,0.0,0.0,106.35,0.0,6.68,0.0,46.09,0.00,0.00,0.00,NaN,75.18
3392,2767,2018,"HARVEST ALASKA, LLC",Crude Oil,100.89,0.0,0.0,0.00,0.0,30.51,0.0,37.24,12.16,0.37,1.42,10.71,111.60
3196,31778,2010,ENCANA OIL & GAS (USA) INC,HVL,NaN,0.0,0.0,0.00,0.0,0.00,0.0,0.00,16.70,0.00,0.00,NaN,0.00
2663,32044,2013,BP USFO/LOGISTICS,Refined and/or Petroleum Product (non-HVL),4.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,NaN,4.00


## 3.3 Extract relevant columns of the pipeline system dataset (2004)

In [22]:
pipelines_2004_2009 = [file for file in listdir('../data/pipelines_2004_2009_2019-08-02/') if 'annual_hazardous_liquid' in file]
pipelines_2004_2009 = pd.concat([pd.read_excel(f'../data/pipelines_2004_2009_2019-08-02/{file}') 
                                 for file in pipelines_2004_2009])
pipelines_2004_2009 = pipelines_2004_2009.reset_index(drop = True)

pipelines_2004_2009.sample(5)

,ORIGINAL,SUPPLEMENT,YR,RPTID,HL_SUP_ID,SYSTEM_TYPE,OPERATOR_ID,NAME,PARENT,OFSTREET,...,BA_5,BA_6,BA_7,BA_8,DOR,DOE,PNAME,PPHONE,PEMAIL,PFAX
123,Y,N,2008,20080300,5086,CRUDE OIL,4906,EXXONMOBIL PIPELINE CO,NaN,"800 BELL STREET, ROOM 623F",...,135.0,75.0,105.0,NaN,2009-06-12,2009-06-12 09:56:55,THAD MASSENGALE,7136562258,THAD.MASSENGALE@EXXONMOBIL.COM,7136568232
249,Y,N,2008,20080385,5192,HVLS,22175,"TARGA MIDSTREAM SERVICES, L.P.",NaN,3001 N NE LOOP 323,...,0.0,0.0,0.0,NaN,2009-06-15,2009-06-15 14:42:27,JAMES M. PETERS,9035338302,JPETERS@TARGARESOURCES.COM,9035338238
447,Y,N,2008,20080364,5171,HVLS,31371,BUCKEYE GULF COAST PIPELINE LP,NaN,"5002 BUCKEYE RD, PO BOX 368",...,351.0,7.0,3.0,NaN,2009-06-12,2009-06-12 14:55:04,DONALD E HANKEY,6109044410,DHANKEY@BUCKEYE.COM,6109044545
1080,Y,N,2009,20090045,6319,PETROLEUM & REFINED PRODUCTS,32386,OILTANKING BEAUMONT PARTNERS L.P.,NaN,6275 HWY. 347,...,0.0,0.0,0.0,NaN,2010-03-08,2010-03-08 00:00:00,AARON STRAIN,4098355381,AARON.STRAIN@OILTANKING.COM,4.09833e+09
1404,Y,N,2009,20090421,7419,CRUDE OIL,26134,EXXONMOBIL OIL CORP - WEST COAST,NaN,"800 BELL STREET, ROOM 623F",...,14.0,0.0,0.0,0.0,2010-06-15,2010-06-15 17:00:07,"THAD MASSENGALE, PIPELINE SAFETY ADVISOR",7136562258,THAD.MASSENGALE@EXXONMOBIL.COM,7.13657e+09


### 3.3.1 Clean name column

In [23]:
pipelines_2004_2009['NAME_FIXED'] = np.where(pd.isnull(pipelines_2004_2009['PARENT']), 
                                             pipelines_2004_2009['NAME'], 
                                             pipelines_2004_2009['PARENT'])
pd.isnull(pipelines_2004_2009['NAME_FIXED']).value_counts()

False    2730
Name: NAME_FIXED, dtype: int64

In [24]:
pipelines_2004_2009[['NAME_FIXED', 'NAME', 'PARENT']].sample(5)

,NAME_FIXED,NAME,PARENT
149,KEY WEST PIPELINE CO,KEY WEST PIPELINE CO,NaN
748,CASCADE PIPELINE1,CASCADE PIPELINE1,NaN
229,WEST TEXAS GULF PIPELINE CO,WEST TEXAS GULF PIPELINE CO,NaN
2084,GOLDEN EAGLE,TESORO REFINING & MARKETING CO,GOLDEN EAGLE
1738,"CHEVRON USA, INC.",CHEVRON PRODUCTS COMPANY HAWAII REFINERY,"CHEVRON USA, INC."


### 3.3.2 Select columns

In [25]:
pipelines_2004_selected = pipelines_2004_2009[['OPERATOR_ID', 'YR', 'NAME_FIXED', 'SYSTEM_TYPE', 'HCAONM', 'ERWTM_1',
                                               'ERWTM_2', 'ERWTM_3', 'ERWTM_4', 'ERWTM_5', 'ERWTM_6', 'ERWTM_7',
                                               'ERWTM_8', 'HCAOFFM', 'HCAMT']].copy()
pipelines_2004_selected.dtypes

OPERATOR_ID      int64
YR               int64
NAME_FIXED      object
SYSTEM_TYPE     object
HCAONM         float64
ERWTM_1        float64
ERWTM_2        float64
ERWTM_3        float64
ERWTM_4        float64
ERWTM_5        float64
ERWTM_6        float64
ERWTM_7        float64
ERWTM_8        float64
HCAOFFM        float64
HCAMT          float64
dtype: object

In [26]:
pipelines_2004_selected['OPERATOR_ID'] = pipelines_2004_selected['OPERATOR_ID'].astype(str)
pipelines_2004_selected.dtypes

OPERATOR_ID     object
YR               int64
NAME_FIXED      object
SYSTEM_TYPE     object
HCAONM         float64
ERWTM_1        float64
ERWTM_2        float64
ERWTM_3        float64
ERWTM_4        float64
ERWTM_5        float64
ERWTM_6        float64
ERWTM_7        float64
ERWTM_8        float64
HCAOFFM        float64
HCAMT          float64
dtype: object

In [27]:
pipelines_2004_selected = pipelines_2004_selected.rename(
    columns={'YR': 'YEAR', 'NAME_FIXED': 'NAME', 'HCAONM': 'MILES', 'HCAOFFM': 'OFFSHORE_MILES', 
             'HCAMT': 'TOTAL_MILES', 'SYSTEM_TYPE': 'COMMODITY', 
             'ERWTM_1': 'MILES_PRE_1940', 
             'ERWTM_2': 'MILES_1940',
             'ERWTM_3': 'MILES_1950',
             'ERWTM_4': 'MILES_1960',
             'ERWTM_5': 'MILES_1970',
             'ERWTM_6': 'MILES_1980',
             'ERWTM_7': 'MILES_1990',
             'ERWTM_8': 'MILES_2000'})
pipelines_2004_selected['MILES_2010'] = 0.0
pipelines_2004_selected.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,OFFSHORE_MILES,TOTAL_MILES,MILES_2010
780,32062,2006,IMTT-PIPELINE,PETROLEUM & REFINED PRODUCTS,7.72,0.0,2.68,0.0,0.00,0.0,0.0,0.0,0.0,2.6,10.32,0.0
1998,22465,2004,WESTERN GAS RESOURCES,HVLS,0.00,26.0,0.00,0.0,0.00,0.0,0.0,31.0,0.0,0.0,0.00,0.0
1639,31339,2005,HUNTSMAN CORP,HVLS,0.00,0.0,0.00,0.0,10.03,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2050,22175,2004,"DYNEGY MIDSTREAM SERVICES, LP",HVLS,1.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.00,0.0
37,31506,2008,"CONOCO, INC., NATURAL GAS AND GAS PRODUCTS DIV...",HVLS,17.60,0.0,0.00,95.5,0.00,0.0,5.1,0.0,0.0,0.0,17.60,0.0


### 3.3.3 Handle "duplicate" observations

How are the instances of diverging names treated by Pandas?

In [28]:
%%R -i pipelines_2004_selected
glimpse(pipelines_2004_selected)

Observations: 2,730
Variables: 16
$ OPERATOR_ID    <chr> "31336", "4805", "8175", "26302", "32147", "4906", "19…
$ YEAR           <int> 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, …
$ NAME           <chr> "CHEVRON U.S.A. INC", "EXPLORER PIPELINE CO", "INTERST…
$ COMMODITY      <chr> "CRUDE OIL", "PETROLEUM & REFINED PRODUCTS", "PETROLEU…
$ MILES          <dbl> 6.000, 1204.110, 8.000, 13.000, 1629.158, 373.000, 40.…
$ MILES_PRE_1940 <dbl> 0.000, 0.000, 0.000, 0.000, 26.841, 1.000, 0.000, 0.00…
$ MILES_1940     <dbl> 0.000, 0.000, 0.000, 0.000, 119.533, 101.000, 0.000, 0…
$ MILES_1950     <dbl> 0.000, 7.200, 0.000, 0.000, 585.263, 126.000, 0.000, 0…
$ MILES_1960     <dbl> 10.647, 7.800, 11.600, 0.000, 107.873, 342.000, 0.000,…
$ MILES_1970     <dbl> 0.000, 593.200, 0.000, 0.000, 243.374, 43.000, 0.000, …
$ MILES_1980     <dbl> 0.000, 18.300, 0.000, 89.000, 91.639, 45.000, 0.000, 0…
$ MILES_1990     <dbl> 0.000, 20.800, 0.000, 0.000, 110.908, 42.000, 0.000, 0…
$ MILES_2000     <

In [29]:
%%R -i pipelines_2004_selected

pipelines_2004_selected <- pipelines_2004_selected %>%
    group_by(OPERATOR_ID, YEAR, COMMODITY) %>%
# We lose some information by how we create the name column, but since we mostly use the OPERATOR_ID, it's alright.
    summarize(NAME = first(NAME), 
              MILES = sum(MILES), 
              MILES_PRE_1940 = sum(MILES_PRE_1940), 
              MILES_1940 = sum(MILES_1940), 
              MILES_1950 = sum(MILES_1950), 
              MILES_1960 = sum(MILES_1960), 
              MILES_1970 = sum(MILES_1970), 
              MILES_1980 = sum(MILES_1980), 
              MILES_1990 = sum(MILES_1990), 
              MILES_2000 = sum(MILES_2000), 
              MILES_2010 = 0,
              OFFSHORE_MILES = sum(OFFSHORE_MILES), 
              TOTAL_MILES = sum(TOTAL_MILES), 
              AGE_UNKNOWN_MILES = 0)
pipelines_2004_selected <- as.data.frame(pipelines_2004_selected)
    
glimpse(pipelines_2004_selected)

Observations: 2,577
Variables: 17
$ OPERATOR_ID       <chr> "10012", "10012", "10012", "10012", "10012", "10012…
$ YEAR              <int> 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007, 200…
$ COMMODITY         <chr> "HVLS", "PETROLEUM & REFINED PRODUCTS", "HVLS", "PE…
$ NAME              <chr> "KANEB PIPE LINE COMPANY LLC", "KANEB PIPE LINE COM…
$ MILES             <dbl> 1191.000, 1196.000, 1616.000, 530.000, 1758.000, 72…
$ MILES_PRE_1940    <dbl> 0, 22, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ MILES_1940        <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 12.000, 0.000, 1…
$ MILES_1950        <dbl> 0.000, 582.000, 0.000, 582.000, 9.000, 50.000, 140.…
$ MILES_1960        <dbl> 1222.140, 941.000, 1222.000, 661.000, 0.000, 176.00…
$ MILES_1970        <dbl> 709.000, 13.000, 709.000, 13.000, 0.000, 0.000, 1.0…
$ MILES_1980        <dbl> 46.000, 359.000, 46.000, 359.000, 1.000, 204.000, 1…
$ MILES_1990        <dbl> 18.000, 10.000, 18.000, 0.000, 20.000, 16.000, 20.0…
$ MILES_2000      

In [30]:
pipelines_2004_selected = %Rget pipelines_2004_selected
pipelines_2004_selected.sample(5)

,OPERATOR_ID,YEAR,COMMODITY,NAME,MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES,AGE_UNKNOWN_MILES
430,20202,2009,CRUDE OIL,ULTRAMAR INC,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
404,20035,2008,CRUDE OIL,UNOCAL PIPELINE COMPANY,3.67,61.35,0.0,0.0,15.97,0.0,0.0,0.0,0.0,0.0,4.17,7.84,0.0
467,22430,2004,CRUDE OIL,WEST SHORE PIPELINE COMPANY,4.00,0.00,0.0,4.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,4.00,0.0
1509,31580,2009,PETROLEUM & REFINED PRODUCTS,"MAGELLAN TERMINALS HOLDINGS, LP",20.00,0.00,0.0,0.0,0.00,3.0,4.0,0.0,1.0,0.0,0.00,20.00,0.0
2547,99002,2008,PETROLEUM & REFINED PRODUCTS,TAMPA PIPELINE LIMITED PARTNERSHIP,10.50,0.00,0.0,0.0,10.50,0.0,0.0,0.0,0.0,0.0,0.00,10.50,0.0


## 3.4 Merge 2010- and 2004- data

### 3.4.1 Unify commodity names

In [31]:
new_names_2010 = {'Crude Oil': 'crude', 
                  'CO2': 'co2',
                  'Fuel Grade Ethanol (dedicated system)': 'fge', 
                  'HVL': 'hvl',
                  'Refined and/or Petroleum Product (non-HVL)': 'non-hvl'}

pipelines_2010_selected = pipelines_2010_selected.replace({'COMMODITY': new_names_2010})
pipelines_2010_selected.sample(3)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES
38,3535,2017,DOW PIPELINE CO,hvl,39.47,1.05,0.00,0.00,0.00,61.72,42.33,27.65,0.00,36.90,0.00,NaN,39.47
4020,300,2016,"PLAINS PIPELINE, L.P.",crude,2312.33,58.49,57.45,726.02,228.88,663.53,254.51,398.52,483.08,462.37,2017.29,NaN,2312.33
927,31336,2011,CHEVRON U.S.A. INC.,crude,6.00,0.00,0.00,0.00,0.00,10.65,0.00,0.00,0.00,0.00,0.00,17.42,23.42


In [32]:
new_names_2004 = {'CRUDE OIL': 'crude', 
                  'HVLS': 'hvl', 
                  'PETROLEUM & REFINED PRODUCTS': 'non-hvl'}

pipelines_2004_selected = pipelines_2010_selected.replace({'COMMODITY': new_names_2004})
pipelines_2004_selected.sample(3)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES
5074,32258,2015,KINDER MORGAN COCHIN LLC,hvl,187.71,0.0,0.0,0.0,0.25,0.0,262.77,0.0,38.86,0.00,1.39,NaN,187.71
5274,39354,2015,ENBRIDGE OFFSHORE FACILITIES,crude,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,25.69,0.00,0.0,0.00
3643,31719,2018,HILAND PARTNERS HOLDINGS LLC,hvl,10.32,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,8.85,12.95,NaN,10.32


### 3.4.2 Merge

In [33]:
pipelines_2004_selected['YEAR'] = pipelines_2004_selected['YEAR'].astype('int64')
pipelines_2004_selected.dtypes

OPERATOR_ID           object
YEAR                   int64
NAME                  object
COMMODITY             object
MILES                float64
AGE_UNKNOWN_MILES    float64
MILES_PRE_1940       float64
MILES_1940           float64
MILES_1950           float64
MILES_1960           float64
MILES_1970           float64
MILES_1980           float64
MILES_1990           float64
MILES_2000           float64
MILES_2010           float64
OFFSHORE_MILES       float64
TOTAL_MILES          float64
dtype: object

In [34]:
pipelines_2010_selected.dtypes

OPERATOR_ID           object
YEAR                   int64
NAME                  object
COMMODITY             object
MILES                float64
AGE_UNKNOWN_MILES    float64
MILES_PRE_1940       float64
MILES_1940           float64
MILES_1950           float64
MILES_1960           float64
MILES_1970           float64
MILES_1980           float64
MILES_1990           float64
MILES_2000           float64
MILES_2010           float64
OFFSHORE_MILES       float64
TOTAL_MILES          float64
dtype: object

Making some adjustments to make the merging seamless.

In [35]:
pipelines_2004_selected = pipelines_2004_selected[['OPERATOR_ID', 'YEAR', 'NAME', 'COMMODITY', 'MILES', 
                                                   'AGE_UNKNOWN_MILES', 'MILES_PRE_1940', 'MILES_1940', 
                                                   'MILES_1950', 'MILES_1960', 'MILES_1970', 'MILES_1980', 
                                                   'MILES_1990', 'MILES_2000', 'MILES_2010', 'OFFSHORE_MILES', 
                                                   'TOTAL_MILES']]

In [36]:
pre_sample = pd.concat([pipelines_2010_selected, pipelines_2004_selected])
pre_sample.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES
4027,473,2016,ANADARKO PETROLEUM CORP,hvl,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,16.5,26.70,NaN,0.00
2467,26134,2013,EXXONMOBIL OIL CORP - WEST COAST,crude,180.70,124.0,0.0,2.0,45.0,31.0,68.0,61.0,133.0,30.0,0.00,NaN,180.70
1429,31304,2014,"DEVON GAS SERVICES, LP",co2,2.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.51,NaN,2.20
411,32412,2017,"MARKWEST LIBERTY MIDSTREAM & RESOURCES, LLC",hvl,164.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,426.51,NaN,164.86
3736,32346,2018,"TOTAL PETROCHEMICALS PIPELINE USA , INC.",hvl,11.00,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.00,NaN,11.00


## 3.5 Calculate percentage offshore and average age

In [48]:
def calc_avg_age(df):
    avg_age = ((df['MILES_PRE_1940'] * 90 + 
                df['MILES_1940'] * 75 + 
                df['MILES_1950'] * 65 + 
                df['MILES_1960'] * 55 + 
                df['MILES_1970'] * 45 + 
                df['MILES_1980'] * 35 + 
                df['MILES_1990'] * 25 + 
                df['MILES_2000'] * 15 + 
                df['MILES_2010'] * 5) /
               (df['MILES_PRE_1940'] + df['MILES_1940'] + df['MILES_1950'] + df['MILES_1960'] + 
                df['MILES_1970'] + df['MILES_1980'] + df['MILES_1990'] + df['MILES_2000'] + df['MILES_2010']))
    return avg_age

In [49]:
pre_sample['AVG_AGE'] = calc_avg_age(pre_sample)
pre_sample['PERC_OFFSHORE'] = pre_sample['OFFSHORE_MILES'].fillna(0.0) / (pre_sample['TOTAL_MILES'].fillna(0.0) + 0.1)


pre_sample.sample(3)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES,AVG_AGE,PERC_OFFSHORE,OFFSHORE_MAX
576,39358,2017,"VELOCITY CENTRAL OKLAHOMA PIPELINE, LLC",crude,23.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,108.4,NaN,23.30,5.00000,0.0,0.0
3536,30777,2018,MOTIVA ENTERPRISE LLC,hvl,35.32,0.0,0.0,0.0,35.3,0.0,0.0,0.0,0.02,0.0,0.0,NaN,35.32,64.97735,0.0,0.0
1897,20035,2012,UNOCAL PIPELINE COMPANY,crude,2.70,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.00,0.0,0.0,0.0,2.70,55.00000,0.0,0.0


## 3.6 Filter commodities and remove offshore operators

### 3.6.1 Commodities

In [50]:
pre_sample = pre_sample[pre_sample['COMMODITY'].isin(['crude', 'hvl', 'non-hvl'])]
pre_sample.sample(3)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES,AVG_AGE,PERC_OFFSHORE,OFFSHORE_MAX
2216,32455,2012,"HAWTHORN OIL TRANSPORTATION (OKLAHOMA), INC",crude,8.750,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.41,NaN,8.750,5.000000,0.0,0.0
2520,31130,2013,DCP MIDSTREAM,hvl,711.552,50.77,0.0,38.78,402.80,260.96,456.47,44.36,357.02,216.46,1179.67,NaN,711.552,28.279125,0.0,0.0
3049,30544,2010,OCCIDENTAL CHEMICAL CORP,non-hvl,15.000,0.00,0.0,0.00,32.87,0.00,0.00,0.00,0.00,0.00,0.00,NaN,15.000,65.000000,0.0,0.0


### 3.6.2 Offshore operators

In [51]:
# For the operators, we remove only those segments (commodities) that have a share of offshore. 
pre_sample['OFFSHORE_MAX'] = (pre_sample['PERC_OFFSHORE'].
                              groupby([pre_sample['OPERATOR_ID'], 
                              pre_sample['COMMODITY']]).transform('max'))
pre_sample.sample(3)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES,AVG_AGE,PERC_OFFSHORE,OFFSHORE_MAX
4446,32428,2016,"INTERCONTINENTAL TERMINAL COMPANY, LLC",hvl,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,0.00,NaN,0.0,0.0
479,38930,2017,"PATOKA TERMINAL COMPANY, LLC",crude,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.45,NaN,0.00,5.0,0.0,0.0
400,32334,2017,TC OIL PIPELINE OPERATIONS INC,crude,787.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,NaN,787.54,NaN,0.0,0.0


In [ ]:
%%R -i pre_sample


In [60]:
distribution = pre_sample.groupby(['OPERATOR_ID', 'COMMODITY'])['PERC_OFFSHORE'].transform('max')
distribution.round(1).value_counts()

0.0    9382
1.0     248
0.1     112
0.9      50
0.6      36
0.2      34
0.4      32
0.3      32
0.7      28
0.8      16
Name: PERC_OFFSHORE, dtype: int64

In [54]:
distribution = pre_sample['PERC_OFFSHORE'].groupby([pre_sample['OPERATOR_ID'], pre_sample['COMMODITY']]).transform('max')
distribution.round(1).value_counts()

0.0    9382
1.0     248
0.1     112
0.9      50
0.6      36
0.2      34
0.4      32
0.3      32
0.7      28
0.8      16
Name: PERC_OFFSHORE, dtype: int64

In [56]:
distribution[distribution < 0.1].round(2).value_counts()

0.00    9314
0.08      36
0.09      24
0.04      18
0.02      18
0.03      18
0.01      14
Name: PERC_OFFSHORE, dtype: int64

In [58]:
len(pre_sample.groupby(['OPERATOR_ID', 'COMMODITY'])['MILES'].sum())

863

### 3.2.3 Filter onshore

In [25]:
# For the operators, we remove only those segments (commodities) that have a share of offshore. 
pipelines_2010_selected['OFFSHORE_MAX'] = (pipelines_2010_selected['PERC_OFFSHORE'].
                                           groupby([pipelines_2010_selected['OPERATOR_ID'], 
                                                    pipelines_2010_selected['COMMODITY']]).transform('max'))
pipelines_2010_selected.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,OFFSHORE_MILES,TOTAL_MILES,AVG_AGE,PERC_OFFSHORE,OFFSHORE_MAX
3844,39122,2018,UTICA EAST OHIO MIDSTREAM LLC,HVL,2.90,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,62.8,NaN,2.90,5.000000,0.0,0.0
4182,26099,2016,TAMPA BAY PIPELINE CO.,HVL,92.80,0.0,0.0,0.0,0.0,0.0,45.82,28.99,1.19,20.75,1.5,NaN,92.80,34.860560,0.0,0.0
3353,39014,2010,"GALVESTON BAY ENERGY, LLC",Crude Oil,NaN,13.2,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,NaN,0.00,NaN,0.0,13.2
3510,26103,2018,"TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO",HVL,116.26,0.0,0.0,0.0,0.0,183.0,0.00,218.10,0.00,0.00,1.6,NaN,116.26,43.969456,0.0,0.0
1441,31443,2014,"ALON USA, LP",Refined and/or Petroleum Product (non-HVL),12.60,0.0,0.0,0.0,0.0,0.0,42.00,0.00,0.00,0.00,0.0,NaN,12.60,45.000000,0.0,0.0


In [26]:
len(pipelines_2010_selected)

5294

In [27]:
pipelines_2010_selected = pipelines_2010_selected.loc[pipelines_2010_selected['OFFSHORE_MAX'] == 0.0].reset_index(drop=True)
pipelines_2010_selected = pipelines_2010_selected.drop(columns=
                                                       ['OFFSHORE_MILES', 'TOTAL_MILES', 'OFFSHORE_MAX'])
pipelines_2010_selected.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,AVG_AGE,PERC_OFFSHORE
2351,31059,2013,BASF CORPORATION,HVL,5.10,0.0,0.0,0.0,0.00,0.0,0.24,7.24,0.00,0.0,0.00,35.320856,0.0
3177,7063,2018,HARBOR PIPELINE CO,Refined and/or Petroleum Product (non-HVL),78.41,0.0,0.0,0.0,77.36,0.0,0.00,0.00,0.00,0.0,3.03,62.738525,0.0
4573,30909,2015,TRANSMONTAIGNE OPERATING COMPANY L.P.,Refined and/or Petroleum Product (non-HVL),66.44,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,NaN,0.0
2343,31023,2013,CITGO REFINING & CHEMICAL CO. L.P.,Crude Oil,7.60,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,NaN,0.0
0,300,2017,"PLAINS PIPELINE, L.P.",HVL,72.90,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.01,0.0,47.92,5.004173,0.0


In [28]:
len(pipelines_2010_selected)

4957

In [29]:
len(np.unique(pipelines_2010_selected['OPERATOR_ID']))

667

### 3.3.3 Calculate on/offshore, age

In [36]:
pipelines_2004_selected['AVG_AGE'] = calc_avg_age(pipelines_2004_selected)

In [37]:
pipelines_2004_selected['PERC_OFFSHORE'] = (pipelines_2004_selected['MILES_OFFSHORE'].fillna(0.0) / 
                                            (pipelines_2004_selected['MILES_TOTAL'].fillna(0.0) + 0.1))
# Function somehow does not work (does not divide ?!)
# calc_perc_offshore(pipelines_2004_selected['MILES_OFFSHORE'], pipelines_2004_selected['ON_AND_OFFSHORE'])
pipelines_2004_selected = pipelines_2004_selected.drop(columns = ['MILES_OFFSHORE', 'MILES_TOTAL'])
pipelines_2004_selected.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,AVG_AGE,PERC_OFFSHORE
2499,30666,2007,"ENMARK ENERGY, INC",CO2 OR OTHER,70.000,0.0,0.000,0.000,0.00,5.000,47.0,0.0,18.0,0.0,30.571429,0.0
1113,31897,2009,"THE GEORGE R. BROWN PARTNERSHIP, L.P.",CO2 OR OTHER,0.000,0.0,0.000,0.000,0.00,0.000,0.0,0.0,7.7,0.0,15.000000,0.0
368,22430,2008,WEST SHORE PIPELINE CO,PETROLEUM & REFINED PRODUCTS,462.726,0.0,33.532,197.464,275.15,3.184,0.0,0.0,0.0,0.0,60.131133,0.0
1520,30829,2005,"TTEPPCO CRUDE PIPELINE, L.P.",CRUDE OIL,336.000,1.0,28.000,15.000,0.00,7.000,0.0,28.0,0.0,0.0,52.911392,0.0
95,31455,2008,CONNACHER OIL AND GAS LTD.,CRUDE OIL,2.600,0.0,0.000,0.000,0.00,0.000,0.0,0.0,2.6,0.0,15.000000,0.0


## 3.4 Save results

In [38]:
pipelines_2004_selected.to_feather(f'../preprocessed_data/pipelines_2004_selected_{today}.feather')

In [39]:
pipelines_2010_selected.to_feather(f'../preprocessed_data/pipelines_2010_selected_{today}.feather')

## 3.4.1 Write original data to .feather for reference

Some columns get erroneously read to data type 'O'. We convert those manually to str type.

In [40]:
pipelines_2010_present.loc[:, pipelines_2010_present.dtypes == 'O'] = pipelines_2010_present.loc[
    :, pipelines_2010_present.dtypes == 'O'].astype(str)

pipelines_2004_2009.loc[:, pipelines_2004_2009.dtypes == 'O'] = pipelines_2004_2009.loc[
    :, pipelines_2004_2009.dtypes == 'O'].astype(str)

incidents.loc[:, incidents.dtypes == 'O'] = incidents.loc[
    :, incidents.dtypes == 'O'].astype(str)

In [41]:
pipelines_2010_present.to_feather(f'../data/pipelines_2010_{today}.feather')
pipelines_2004_2009.to_feather(f'../data/pipelines_2004_{today}.feather')
incidents.to_feather(f'../data/incidents_{today}.feather')